In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
pd.options.display.max_columns = None
from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix, precision_score, recall_score,  accuracy_score, precision_recall_curve
import seaborn as sns
import nltk
import re
import tensorflow as tf
import keras 
from sklearn.model_selection import train_test_split
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers import SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.metrics import confusion_matrix


In [7]:

data = pd.read_csv(r"/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv",delimiter=',',engine='python',encoding='utf-8-sig')


In [8]:
data.loc[data["class"] == 0, "class"] = 1 
data.loc[data["class"]== 2, "class"] = 0

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('all')
import string
stemmer = nltk.SnowballStemmer('english')
stopword = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading all: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [10]:
def data_cleaning(words):
    words = str(words).lower()
    words = re.sub('@[\w]+','',words)
    words = re.sub('\[.*?/]', "", words)
    words = re.sub('https?://\S+|www\.\S+', "", words)
    words = re.sub('<.*?=+->^$#@!%', "", words)
    words = re.sub('\n', "", words)
    words = [word for word in words if word not in string.punctuation]
    words = "".join(words)
    words = [word for word in words.split(' ') if word not in stopword]
    words = " ".join(words)
    words = [stemmer.stem(word) for word in words.split(' ')]
    words = " ".join(words)
    
    return words

In [11]:
data['tweet'] = data['tweet'].apply(data_cleaning)
data['tweet'].head(10)

0     rt  woman shouldnt complain clean hous amp ma...
1     rt  boy dat coldtyga dwn bad cuffin dat hoe 1...
2     rt  dawg rt  ever fuck bitch start cri confus...
3                                rt   look like tranni
4     rt  shit hear might true might faker bitch to...
5     shit blow meclaim faith somebodi still fuck h...
6               sit hate anoth bitch  got much shit go
7    8220 caus im tire big bitch come us skinni gir...
8                amp might get ya bitch back amp that 
9                       hobbi includ fight mariambitch
Name: tweet, dtype: object

In [12]:
x = data['tweet']
y = data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 50000
max_len = 300

tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(x_train)

sequences = tokenizer.texts_to_sequences(x_train)
sequences_matrix = pad_sequences(sequences, maxlen = max_len)

In [15]:
from tensorflow.keras.models import Sequential
from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.layers import SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from sklearn.metrics import confusion_matrix


In [16]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_shape=(300,)))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 300, 100)       │     5,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ (None, 300, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100)            │        80,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,080,501 (19.38 MB)

 Trainable params: 5,080,501 (19.38 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(sequences_matrix, y_train, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 75s 586ms/step - accuracy: 0.8305 - loss: 0.4170 - val_accuracy: 0.9317 - val_loss: 0.1637
Epoch 2/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 72s 581ms/step - accuracy: 0.9448 - loss: 0.1408 - val_accuracy: 0.9476 - val_loss: 0.1344
Epoch 3/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 74s 597ms/step - accuracy: 0.9644 - loss: 0.0987 - val_accuracy: 0.9476 - val_loss: 0.1312
Epoch 4/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 73s 588ms/step - accuracy: 0.9712 - loss: 0.0808 - val_accuracy: 0.9455 - val_loss: 0.1338
Epoch 5/5
124/124 ━━━━━━━━━━━━━━━━━━━━ 82s 585ms/step - accuracy: 0.9779 - loss: 0.0716 - val_accuracy: 0.9473 - val_loss: 0.1398


In [18]:
test_sequences = tokenizer.texts_to_sequences(x_test)
test_sequences_matrix = pad_sequences(test_sequences, maxlen=max_len )

In [19]:
acc = model.evaluate(test_sequences_matrix, y_test)

155/155 ━━━━━━━━━━━━━━━━━━━━ 11s 69ms/step - accuracy: 0.9472 - loss: 0.1541


In [20]:
lstm_prediicton = model.predict(test_sequences_matrix)

155/155 ━━━━━━━━━━━━━━━━━━━━ 11s 67ms/step


In [21]:
res = []
for prediction in lstm_prediicton:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [22]:
print(confusion_matrix(y_test,res))

[[ 757   99]
 [ 148 3953]]


In [23]:
model.save("model.h5")

In [24]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
import tensorflow.keras as keras
load_model=keras.models.load_model("model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [26]:
test = 'you are an idiot'

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

test=[clean_text(test)]
print(test)

seq = load_tokenizer.texts_to_sequences(test)
padded = pad_sequences(seq, maxlen=300)
print(seq)

pred = load_model.predict(padded)

print("pred", pred)
if pred<0.5:
    print("not offensive")
else:
    print("offensive")
    

you are an idiot
you are an idiot
['idiot']
[[1293]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step
pred [[0.65941143]]
offensive


EXPLANATION